# Counterparty Credit Risk & Backtesting MVP

**Head of CCR Level Demo** – Pure Python | Zero Paid Services | Runs anywhere

As-of date: **25 Nov 2025** | 2,500 Monte Carlo paths | 95% PFE

In [ ]:
import pandas as pd
from core.exposure import CCRSimulator
import matplotlib.pyplot as plt
%matplotlib inline

trades = pd.read_csv('../data/sample_portfolio.csv', parse_dates=['maturity_date'])
sim = CCRSimulator(n_paths=2500)
profiles = sim.run_netting_set(trades)

print(f"Successfully simulated {len(profiles)} netting sets")
profiles['NS001'].head()

## Exposure Profile – NS001 (Deutsche Bank – Investment Grade)

In [ ]:
df = profiles['NS001']
plt.figure(figsize=(12,6))
plt.plot(df['time'], df['EE_total']/1e6, label='Expected Exposure (EE)', linewidth=3, color='#2E8B57')
plt.plot(df['time'], df['PFE_total']/1e6, label='PFE 95%', linewidth=3, color='#DC143C')
plt.title('Netting Set NS001 – Deutsche Bank (FX Forward + IRS)', fontsize=16)
plt.xlabel('Time Horizon (years)', fontsize=12)
plt.ylabel('Exposure (€ millions)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## Limits & EWI Dashboard

In [ ]:
from analytics.limits_ewi import apply_limits_and_ewi
from analytics.backtest import run_historical_backtest

dashboard = apply_limits_and_ewi(profiles, trades)
print("Limits & EWI Summary")
display(dashboard.style.apply(lambda x: ['background: #ffcccc' if v == 'BREACH' else '' for v in x], subset=['Status']))

## Backtesting Result (2018–2024) – BIS Coverage Test

In [ ]:
bt = run_historical_backtest()
result_df = pd.DataFrame([bt])
display(result_df.style.apply(lambda x: ['background: lightgreen' if v else 'background: #ffcccc' for v in x], subset=['Pass']))